In [1]:
import torch,os,csv
import sys
sys.path.append('xxx/Hotels-50K/evaluate/')
from evaluate.retrieval import main

from _code.Reader import ImageReader
from _code.Utils import eva_transforms
from _code.color_lib import RGBmean, RGBstdv


def Acc(epoch_idx,case):
    src = '_result/HOTEL_R50/'+case
    print(src)
    acc = list()
    for ep in epoch_idx:
        print('epoch:'+str(ep))
        data_dict = torch.load('data_dict_emb.pth')
        
        testdset = ImageReader(data_dict['test'], eva_transforms(256, RGBmean['HOTEL'], RGBstdv['HOTEL']))
        tradset = ImageReader(data_dict['tra'], eva_transforms(256, RGBmean['HOTEL'], RGBstdv['HOTEL']))

        query_idx = torch.load(src+'{}pre100.pth'.format(ep))

        test_id = [os.path.basename(testdset.imgs[i][0])[:-4] for i in range(len(testdset))]
        result_id = [[os.path.basename(tradset.imgs[query_idx[i,j].item()][0])[:-4] for j in range(100)] for i in range(len(valdset))]

        outfile = os.path.join('xxx/Hotels-50K/nnloss_results','hotel_results_'+str(ep)+'.csv')
        with open(outfile,'w') as csv_file:
            for idx in range(len(test_id)):
                imId = test_id[idx]
                result_im_ids = result_id[idx]
                csv_line = str(imId) + ',' + ','.join([str(r) for r in result_im_ids]) +'\n'
                csv_file.writelines(csv_line)

        acc.append(main(outfile))
    
    return acc

epoch_idx = [i for i in range(20,205,20)]
case = 'EPSHN_LR_1e-02_ep100/G2/'
Acc(epoch_idx,case)